### Edgar Moises Hernandez-Gonzalez
#### Asesores: Pilar Gomez-Gil, Erik Bojorges-Valdez
#### Instituto Nacional de Astrofísica Óptica y Electrónica (INAOE)
#### 26/11/20-03/12/20
#### Tesis: Clasificación de señales EEG basada en representaciones bidimensionales y redes neuronales convolucionales
#### Clasificacion de EEG con CNN-2D o CNN-2D + LSTM
##### Caracteristicas = Espectrogramas STFT o Escalogramas CWT

##### Predecir la etiqueta de nuevos ejemplos de EEG

In [1]:
# activar google drive
# no es necesario si se ejecuta local
from google.colab import drive

In [2]:
# montar drive
# no es necesario si se ejecuta local
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# ejecutar desde aqui si se ejecuta en su computadora local, importar librerias
import numpy as np
import pandas as pd
import time
from scipy.signal import spectrogram
import pywt
import cv2
from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix
from keras.models import load_model

In [4]:
# no necesitamos y_train, solo vamos a predecir
# leer .csv con los datos de train y test
# se debe especificar la ruta
# header = None significa que los .csv no tienen encabezado
x_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2b/Datos/MI-EEG-B9T.csv",
                      header=None)
x_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2b/Datos/MI-EEG-B9E.csv",
                     header=None)
y_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2b/Datos/etiquetas_test_9.csv",
                     header=None)

In [5]:
# imprimir la forma de las matrices de datos
# x_train y x_test son matrices donde las filas son el numero de ejemplos
# y las columnas son el numero de segundos por la frecuencia de muestreo
# por el numero de canales
# y_train y y_test son las etiquetas
print(x_train.shape)
print(x_test.shape)
print(y_test.shape)

(400, 3000)
(320, 3000)
(320, 1)


In [6]:
# calcular el numero de clases
# esto se podria calcular asi n_clases = len(np.unique(y))
# la y puede ser train o test
n_classes = len(np.unique(y_test))

print("n_classes:", n_classes)

n_classes: 2


In [7]:
# calcular espectrogramas con STFT
# concatenacion vertical de los espectrogramas de los n canales
# unir_espectrogramas_vertical(matriz de x_train o x_test,
# frecuencia de muestreo, alto, ancho, n_canales, tamano del segmento de senal,
# puntos superpuestos para calcular la STFT)
def unir_espectrogramas_vertical(data, fs, alto, ancho, n_canales, pts_sig,
                                 pts_superpuestos):
  #fs = fs #frecuencia de muestreo
  # crear matriz 3D para almacenar todas las imagenes de los STFT
  datos = np.zeros((data.shape[0],alto, ancho))
  
  # crear matriz 2D donde se guardara cada imagen del STFT
  temporal = np.zeros((alto, ancho))

  for i in range(data.shape[0]): # n muestras
    for j in range(n_canales): # n canales
      # puntos_sig = duracion en segundos de la señal x frecuencia de muestreo
      # para una señal de 2 seg con fs de 250 Hz, puntos_sig = 500
      sig = data.iloc[i, j*pts_sig:(j+1)*pts_sig]
      
      # espectrograma con STFT
        # senal,
        # fs=frecuencia de muestreo,
        # window='tipo de ventana',
        # nperseg=tamano de ventana,
        # noverlap=puntos superpuestos,
        # nfft=tamano de la FFT con relleno de ceros, Si=None la longitud de FFT es nperseg
        # scaling=‘density’: power spectral density, ‘spectrum’: power spectrum
      f, t, Sxx = spectrogram(sig, fs=fs, window='hann', nperseg=fs,
                              noverlap=pts_superpuestos, nfft=fs*2,
                              scaling='spectrum')
      
      # concatenacion vertical de canales
      # espectrograma genera 45 filas (frecuencias de 8 a 30Hz con pasos de
      # 0.5Hz) y 31 columnas (tiempo)
      # las frecuncias de 8 a 30 Hz estan entre los indices de 16 a 60
      temporal[j*45:(j+1)*45, :] = Sxx[16:61, :]

    datos[i] = temporal
    if i % 100 == 0: # esto es para ver como avanza
      print(i)
  return datos

In [8]:
# calcular escalogramas con CWT
# concatenacion vertical de los escalogramas de los n canales
# unir_escalogramas_vertical(matriz de x_train o x_test, frecuencia de muestreo,
# alto, ancho, n_canales, tamano del segmento de senal):
def unir_escalogramas_vertical(data, fs, alto, ancho, n_canales, pts_sig):
  # ancho y alto, para el resize usando Open CV
  dim = (int(np.floor(ancho/2)), int(np.floor(alto/2))) # ancho, alto
  
  # calcular escalas para Wavelet compleja de Morlet 3-3
  # con frecuencias de 8 a 30 Hz con pasos de 0.5 Hz
  escalas = pywt.scale2frequency('cmor3-3', np.arange(8,30.5,0.5)) / (1/fs)
  
  # crear matriz 3D para almacenar todas las imagenes
  datos = np.zeros((data.shape[0], int(np.floor(alto/2)),
                    int(np.floor(ancho/2))))
  
  # crear matriz 2D donde se guardara cada imagen
  temporal = np.zeros((alto, ancho))

  for i in range(data.shape[0]): # n muestras
    for j in range(n_canales): # n canales
      # puntos_sig = duracion en segundos de la señal x frecuencia de muestreo
      # para una señal de 2 seg con fs de 250 Hz, puntos_sig = 500
      sig = data.iloc[i, j*pts_sig:(j+1)*pts_sig]
      
      # escalograma con CWT
        # senal
        # escalas
        # nombre de la wavelet, en este caso compleja de Morlet 3-3
        # periodo de muestreo = 1 / frecuencia de muestreo
      coef, freqs = pywt.cwt(sig, escalas, 'cmor3-3',
                             sampling_period = (1 / fs))
      
      # concatenacion vertical de canales
      # espectrograma genera 45 filas (frecuencias de 8 a 30Hz con pasos de
      # 0.5Hz) y 31 columnas (tiempo)
      # dado que cmor3-3 genera numeros complejos, calcular el modulo
      temporal[j*45:(j+1)*45, :] = abs(coef)

    # resize usando una interpolacion interarea con OpenCV
    resized = cv2.resize(temporal, dim, interpolation=cv2.INTER_AREA)
    datos[i] = resized
    if i % 100 == 0: # esto solo es para ver como avanza
      print(i)
  return datos

In [9]:
# seleccionar STFT o CWT
inicio = time.time()

# STFT, llamar a unir_espectrogramas_vertical(data, fs, alto, ancho,
# n_canales, pts_sig, pts_superpuestos))
# descomentar las siguientes dos lineas para STFT
x_train = unir_espectrogramas_vertical(x_train, 250, 135, 31, 3, 1000, 225)
x_test = unir_espectrogramas_vertical(x_test, 250, 135, 31, 3, 1000, 225)

# CWT, llamar a unir_escalogramas_vertical(data, fs, alto, ancho,
# n_canales, pts_sig)
# descomentar las siguientes dos lineas para CWT
#x_train = unir_escalogramas_vertical(x_train, 250, 135, 1000, 3, 1000)
#x_test = unir_escalogramas_vertical(x_test, 250, 135, 1000, 3, 1000)

fin = time.time()
print("Tiempo:", fin - inicio)

0
100
200
300
0
100
200
300
Tiempo: 2.064129114151001


In [10]:
# forma de los espectrogramas (STFT) o escalogramas (CWT)
# la forma debe ser tridimensional
# la primera dimension es igual al numero de ejemplos
# la segunda dimension es lo alto de las imagenes calculadas con STFT o CWT
# la tercera dimension es lo ancho de las imagenes calculadas con STFT o CWT
print(x_test.shape)

(320, 135, 31)


In [11]:
#convertir a float
x_test = x_test.astype('float32')

# escalar los valores en un rango de 0 a 1 (normalizar)
# dividir entre el max redondeado al techo de x_train
x_test /= np.ceil(np.max(x_train))

In [12]:
# seleccionar reshape a 4D (para CNN-2D) o 5D (para CNN-2D + LSTM)
# para CNN-2D usar a 4D, para CNN-2D + LSTM usar a 5D

# convertir de 3D a 4D (CNN-2D)
# descomentar las siguientes dos lineas para usar CNN-2D
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], x_test.shape[2], 1))

# convertir de 3D a 5D (CNN-2D + LSTM)
# descomentar las siguientes dos lineas para usar CNN-2D + LSTM
#x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1], x_test.shape[2], 1))

# imprimir la forma
print(x_test.shape)

(320, 135, 31, 1)


In [13]:
# indicar la ruta donde se encuentra el modelo entrenado en .hdf5
model = load_model('/content/drive/My Drive/BCI-IV-2b/Modelos/STFT_CNN_09.hdf5')
#model = load_model('/content/drive/My Drive/BCI-IV-2b/Modelos/CWT_RNN_09.hdf5')

In [14]:
# resumen del modelo (red neuronal)
# se muestra la forma de la salida de cada capa y
# el numero de parametros a apreder en cada capa
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 135, 31, 4)        40        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 67, 15, 4)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 67, 15, 4)         148       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 33, 7, 4)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 924)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                29600     
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)               

### Evaluar varios ejemplos test

In [15]:
inicio = time.time()

# evaluar red neuronal con x_test y y_test
# (datos que nunca se le presentaron a la red en el entrenamiento)
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 0.5540986061096191


In [16]:
# este accuracy aveces lo calcula mal (no encontre una explicacion del porque),
# mas abajo se calcula de forma correcta
print("loss: %.4f" % test_loss)
print("accuracy: %.4f" % test_acc)

loss: 0.5589
accuracy: 0.4563


In [17]:
# predecir x_test
# obtener las probabilidades de la capa softmax de la red neuronal
probabilidades = model.predict(x_test)

# obtener las etiquetas predichas calculando el indice del maximo
# de la distribucion de probalidad
y_pred = np.argmax(probabilidades, 1) # 1 = fila

In [18]:
print("accuracy: %.4f" % accuracy_score(y_test, y_pred))

accuracy: 0.8187


In [19]:
print("kappa: %.4f" % cohen_kappa_score(y_test, y_pred))

kappa: 0.6375


In [20]:
print("confusion matrix:\n", confusion_matrix(y_test, y_pred))

confusion matrix:
 [[124  36]
 [ 22 138]]


### Evaluar un ejemplo

In [21]:
# en este ejemplo se predice la etiqueta de una imagen de un STFT de 135x31
# cambiar x_test[0] por el ejemplo a predecir
# x_test[0] es el primer ejemplo de x_test

inicio = time.time()

# para CNN-2D
probabilidades = model.predict(x_test[0].reshape(1,135,31,1))

# para CNN-2D + LSTM
#probabilidades = model.predict(x_test[0].reshape(1,1,67,500,1))

# obtener las etiquetas predichas calculando el indice del maximo
# de la distribucion de probalidad
print(probabilidades.argmax())

fin = time.time()
print("Tiempo:", fin - inicio)

1
Tiempo: 0.13254094123840332
